# Import

In [1]:
from torch import nn
import torch
from models import DrFuse_RS_Model, DrFuse_RS_Trainer
import time
import os
from torch.utils.data import Dataset, DataLoader
from tqdm.autonotebook import tqdm
import skimage.io as io
import earthpy.spatial as es
import numpy as np
import random
import wandb


# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)

C:\Users\n10300759\AppData\Local\Temp\ipykernel_30652\2494540655.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


# Data

In [2]:
now = str(int(time.time()))

EXPERIMENT_CONFIG = dict(
    # window data folder
    DATA_FOLDER = r'D:\CarsSegmentationTroubleShoot\data\multimodal',
    # mac data folder
    # DATA_FOLDER = r'/Users/nhikieu/Documents/PhD/multimodal',
    num_epoch = 100, 
    img_size = 512, 
    num_classes = 6,
    input_channels = 5,
    batch_size = 4,
    checkpoint_pth = os.path.join('checkpoints', now),
    update_optim_size = 32
    )
DEVICE = 'mps'

print(EXPERIMENT_CONFIG['checkpoint_pth'])

checkpoints\1721218346


In [3]:
def rgb_to_class(mask_image):
  class_map = {
    (255,255,255): 0, 
    (0,0,255): 1, 
    (0,255,255): 2, 
    (0,255,0): 3, 
    (255,255,0): 4, 
    (255,0,0): 5
  }

  # Create a 3D numpy array that represents the RGB color of each pixel
  rgb_data = mask_image.reshape(-1, 3)

  # Create a 1D numpy array that represents the class label for each RGB color
  class_labels = np.zeros(rgb_data.shape[0], dtype=np.uint8)
  for rgb, class_label in class_map.items():
      mask = np.all(rgb_data == np.array(rgb), axis=1)
      class_labels[mask] = class_label

  # Reshape the 1D class label array into a 2D class map
  class_data = class_labels.reshape(mask_image.shape[:2])

  return class_data


class VaihingenDataset(Dataset):
  def __init__(self, folder) -> None:
    '''
    folder: data path include both rgb img and gt
    gt: 3 channels map with postfix '_gt'
    '''
    super().__init__()

    self.imgs = []
    self.gts = []

    # get all filenames in the directory
    filenames = os.listdir(os.path.join(folder, 'rgb'))

    for f in tqdm(filenames):
      img = io.imread(os.path.join(folder, 'rgb', f)) / 255.0
     
      ndsm = io.imread(os.path.join(folder, 'ndsm', f.split('.png')[0] + '_ndsm.tif'))
      ndsm = np.expand_dims(ndsm, axis=2)
    
      input_4C = np.dstack((img, ndsm))

      input_4C = torch.tensor(input_4C).permute((2, 0, 1))
      
      gt_path = os.path.join(folder, 'gt', f.split('.png')[0] + '_gt.png')
      gt = io.imread(gt_path)
      gt = rgb_to_class(gt)
      gt = torch.tensor(gt).unsqueeze(0)

      self.imgs.append(input_4C)
      self.gts.append(gt)
    

  def __len__(self):
    return len(self.imgs)
  
  def __getitem__(self, index):
    input_4C = self.imgs[index].float()
    gt = self.gts[index]

    # mask_array = [True, False]
    mask_index = np.random.choice(2, 1, p=[0.5, 0.5])
    mask = torch.tensor(mask_index[0])

    return input_4C, gt, mask

In [4]:
#Note that we are using ground truth in the folder '1CGT'. They are not RGB anymore, but one channel.
training_dataset = VaihingenDataset(os.path.join(EXPERIMENT_CONFIG['DATA_FOLDER'], 'train'))
validate_dataset = VaihingenDataset(os.path.join(EXPERIMENT_CONFIG['DATA_FOLDER'], 'val'))

#Note that we shuffle the data for the training set, but not for the validation set:
train_loader = DataLoader(dataset=training_dataset, batch_size=EXPERIMENT_CONFIG['batch_size'], shuffle=True, pin_memory=True)
validate_loader = DataLoader(dataset=validate_dataset, batch_size=EXPERIMENT_CONFIG['batch_size'], shuffle=True, pin_memory=True)

print(f'Train samples: {len(training_dataset)}')
print(f'Val samples: {len(validate_dataset)}')

100%|██████████| 120/120 [00:20<00:00,  6.00it/s]

Train samples: 1620
Val samples: 120


# Train

In [5]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nhikieu (qut_nhi_phd). Use `wandb login --relogin` to force relogin


True

In [6]:
# best pretrained path
# ckpt_path = r"D:\drfuse\checkpoints\1720700298\1720724395_loss1.8435" # pretrained
# ckpt_path = r"D:\drfuse\checkpoints\1720752266\1720794150_loss16667.4636" # cont shared feats
# ckpt_path = r"D:\drfuse\checkpoints\1721047649\1721088673_loss1759.3356"
ckpt_path = r"D:\drfuse\checkpoints\1721101178\1721127137_loss1164.4341"
with wandb.init(project="drfuse", entity="nhikieu", config=EXPERIMENT_CONFIG):
    config = wandb.config
    model_trainer = DrFuse_RS_Trainer(config, pretrain=False, ckpt=ckpt_path)
    model_trainer(train_loader, validate_loader)

wandb: Currently logged in as: nhikieu. Use `wandb login --relogin` to force relogin


  0%|          | 0/150 [00:00<?, ?it/s]d:\drfuse\models\drfuse_rs.py:88: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  masks = torch.tensor(masks).to(device)
d:\drfuse\drfuse_venv\lib\site-packages\torch\nn\modules\instancenorm.py:80: UserWarning: input's size at dim=0 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
  warnings.warn(f"input's size at dim={feature_dim} does not match num_features. "
d:\drfuse\models\drfuse_rs.py:111: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  masks = torch.tensor(masks).to(device)


Train loss after 00512 examples: 6326.419
Val loss after 00512 examples: 10302.239
Train loss after 01024 examples: 3737.991
Val loss after 01024 examples: 3383.319
Train loss after 01536 examples: 3555.641
Val loss after 01536 examples: 2122.848
Train loss after 01620 examples: 3182.374
Val loss after 01620 examples: 2109.753


  1%|          | 1/150 [06:10<15:20:19, 370.60s/it]

Train loss after 02048 examples: 4933.048
Val loss after 02048 examples: 6188.715
Train loss after 02560 examples: 4532.050
Val loss after 02560 examples: 3756.334
Train loss after 03072 examples: 2509.765
Val loss after 03072 examples: 3438.205


...

Train loss after 240128 examples: 1591.610
Val loss after 240128 examples: 1295.440
Train loss after 240640 examples: 1385.085
Val loss after 240640 examples: 1216.930
Train loss after 241152 examples: 1698.218
Val loss after 241152 examples: 1357.110
Train loss after 241380 examples: 1431.950
Val loss after 241380 examples: 1307.083


 99%|█████████▉| 149/150 [13:27:58<05:25, 325.74s/it]

Train loss after 241664 examples: 1697.526
Val loss after 241664 examples: 1155.366
Train loss after 242176 examples: 1666.876
Val loss after 242176 examples: 1455.578
Train loss after 242688 examples: 1727.435
Val loss after 242688 examples: 1433.504
Train loss after 243000 examples: 1471.023
Val loss after 243000 examples: 1629.528


100%|██████████| 150/150 [13:33:33<00:00, 325.42s/it]


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_loss,▄█▄▃▃▃▃▃▃▁▁▂▂▁▂▁▁▂▃▁▂▃▃▂▂▂▂▂▃▁▂▃▃▁▂▂▁▃▂▃
val_loss,▇█▅▂▂▂▃▂▁▂▃▂▁▁▁▁▂▁▁▁▂▅▄▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▆▂
epoch,149
train_loss,1471.02307
val_loss,1629.52843


# Unit Test

In [3]:
dummies = torch.rand(6, 4, 512, 512)
masks = torch.FloatTensor([True, False, True, True, False, False])
model = DrFuse_RS_Model()
results = model(dummies, masks)
print(results['pred_rgb'].shape, results['pred_ndsm'].shape, 
      results['pred_shared'].shape, results['pred_multimodal'].shape,
      results['aux_preds'][0].shape)

# Assuming model is an instance of a class derived from torch.nn.Module
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total trainable parameters: {total_params}")

d:\drfuse\drfuse_venv\lib\site-packages\torch\nn\modules\instancenorm.py:80: UserWarning: input's size at dim=0 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
  warnings.warn(f"input's size at dim={feature_dim} does not match num_features. "


torch.Size([6, 6, 512, 512]) torch.Size([6, 6, 512, 512]) torch.Size([6, 6, 512, 512]) torch.Size([6, 6, 512, 512]) torch.Size([6, 6, 512, 512])
Total trainable parameters: 42120024


In [ ]:
pairs = torch.FloatTensor([True, False, True, False, True, True])
pairs = pairs.unsqueeze(1)
feat_dummy = torch.randn(6, 512)
test = pairs*feat_dummy
test

In [ ]:
for i, (input_4C, gt, mask) in enumerate(validate_loader):
  print(input_4C.shape, gt.shape, mask.shape)
  print(mask)
  break